In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [6]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary
import time

In [7]:
"""
X_train = torch.randn([14231, 1, 9000, 18])
Y_train = torch.ones([14231, 1])
X_val = torch.randn([1581, 1, 9000, 18])
Y_val = torch.ones([1581, 1])
X_test = torch.randn([10249, 1, 9000, 18])
Y_test = torch.ones([10249, 1])
"""

'\nX_train = torch.randn([14231, 1, 9000, 18])\nY_train = torch.ones([14231, 1])\nX_val = torch.randn([1581, 1, 9000, 18])\nY_val = torch.ones([1581, 1])\nX_test = torch.randn([10249, 1, 9000, 18])\nY_test = torch.ones([10249, 1])\n'

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 18))
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.padding1 = nn.ZeroPad2d(10)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 320, (5, 5), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(320, False)

        # FC
        self.fc1 = nn.Linear(2309120, 1)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = self.padding1(x)
        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)


        #x = x.permute(0, 3, 1, 2)



        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x

if __name__ == "__main__":
    model = CNN().cuda()
    summary(model.cuda(), (1, 9000, 18))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 9000, 1]             304
       BatchNorm2d-2          [-1, 16, 9000, 1]              32
         ZeroPad2d-3         [-1, 16, 9020, 21]               0
            Conv2d-4         [-1, 320, 1804, 4]         128,320
       BatchNorm2d-5         [-1, 320, 1804, 4]             640
            Linear-6                    [-1, 1]       2,309,121
Total params: 2,438,417
Trainable params: 2,438,417
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.62
Forward/backward pass size (MB): 60.55
Params size (MB): 9.30
Estimated Total Size (MB): 70.47
----------------------------------------------------------------
